In [ ]:
-- Se eliminan las tablas para un entorno limpio (opcional)
DROP TABLE IF EXISTS detalle_ventas CASCADE;
DROP TABLE IF EXISTS ventas CASCADE;
DROP TABLE IF EXISTS productos CASCADE;
DROP TABLE IF EXISTS clientes CASCADE;

-- Se crean las tablas
CREATE TABLE clientes (
    cliente_id SERIAL PRIMARY KEY,
    nombre VARCHAR(100),
    apellido VARCHAR(100),
    email VARCHAR(100),
    telefono VARCHAR(50)
);

CREATE TABLE productos (
    producto_id SERIAL PRIMARY KEY,
    nombre_producto VARCHAR(100),
    categoria VARCHAR(100),
    precio NUMERIC(6,2)
);

CREATE TABLE ventas (
    venta_id SERIAL PRIMARY KEY,
    cliente_id INT REFERENCES clientes (cliente_id),
    fecha DATE,
    total NUMERIC(10,2)
);

CREATE TABLE detalle_ventas (
    detalle_venta_id SERIAL PRIMARY KEY,
    venta_id INT REFERENCES ventas (venta_id),
    producto_id INT REFERENCES productos (producto_id),
    cantidad INT,
    subtotal NUMERIC(10,2)
);

-- Se fija la semilla aleatoria para reproducir resultados
SELECT setseed(0.5);

-- Se insertan los clientes
INSERT INTO clientes (nombre, apellido, email, telefono)
SELECT 
    'Cliente_' || id AS nombre,
    'Apellido_' || id AS apellido,
    'cliente' || id || '@example.com' AS email,
    '123-456-' || LPAD(id::TEXT, 4, '0') AS telefono
FROM generate_series(1, 100) AS id;

-- Se insertan los productos
INSERT INTO productos (nombre_producto, categoria, precio)
SELECT 
    'Producto_' || id AS nombre_producto,
    CASE (id % 4) + 1
        WHEN 1 THEN 'Lácteos'
        WHEN 2 THEN 'Carnicería'
        WHEN 3 THEN 'Frutas'
        WHEN 4 THEN 'Panadería'
    END AS categoria,
    ROUND((RANDOM() * 20 + 1)::NUMERIC, 2) AS precio
FROM generate_series(1, 100) AS id;

-- Se insertan las ventas
-- Solo se usan IDs de clientes existentes para evitar errores de referencia
INSERT INTO ventas (cliente_id, fecha, total)
SELECT 
    c.cliente_id,
    CURRENT_DATE - (RANDOM() * 30)::INT AS fecha,
    ROUND((RANDOM() * 200 + 10)::NUMERIC, 2) AS total
FROM generate_series(1, 100) AS g
CROSS JOIN (
    SELECT cliente_id 
    FROM clientes
) c
ORDER BY RANDOM()
LIMIT 100;

-- Se insertan detalles de ventas
-- Se limita a 200 filas para repartir productos de manera aleatoria
INSERT INTO detalle_ventas (venta_id, producto_id, cantidad, subtotal)
SELECT 
    v.venta_id,
    p.producto_id,
    (RANDOM() * 5 + 1)::INT AS cantidad,
    ROUND(p.precio * (RANDOM() * 5 + 1)::NUMERIC, 2) AS subtotal
FROM ventas v
CROSS JOIN productos p
ORDER BY RANDOM()
LIMIT 200;